<a href="https://colab.research.google.com/github/MECHTI1/ZS/blob/master/ZinxSightv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Environment Setup and Imports

# Install Essential System Packages and GPG Tools
!sudo apt install dirmngr ca-certificates software-properties-common gnupg gnupg2 apt-transport-https curl -y

# Install PostgresSQL
!curl -fSsL https://www.postgresql.org/media/keys/ACCC4CF8.asc | gpg --dearmor | sudo tee /usr/share/keyrings/postgresql.gpg > /dev/null
!echo 'deb [arch=amd64,arm64,ppc64el signed-by=/usr/share/keyrings/postgresql.gpg] http://apt.postgresql.org/pub/repos/apt/ jammy-pgdg main' | sudo tee /etc/apt/sources.list.d/pgdg.list
!sudo apt update
!sudo apt install postgresql-client-16 postgresql-16 -y
!sudo service postgresql start

# Create a new PostgreSQL database if it does not already exist
!sudo -u postgres psql -tc "SELECT 1 FROM pg_database WHERE datname = 'zincsight_pipeline_db';" | grep -q 1 || sudo -u postgres psql -c "CREATE DATABASE zincsight_pipeline_db;"
!sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE zincsight_pipeline_db TO postgres;"

# Install aria2 for optional batch downloads
!apt install aria2 -y

# Clone the GitHub repository and change into the repo directory
!git clone git@github.com:MECHTI1/ZS.git
%cd /content/ZS/

# Add essential SQL tables to the established database
!sudo -u postgres psql -d zincsight_pipeline_db -f "src/setup_pg_db_with_tables/PostgreSQL_4_necessary_tables.sql"

# Install libraries or dependencies
!pip install -r requirements.txt


In [ ]:
%pip install -q ipywidgets
import ipywidgets as widgets
from IPython.display import display
import os

In [ ]:
mounted_to_drive=True

if mounted_to_drive:
  # Mount Google Drive
  from google.colab import drive
  drive.mount('/content/drive/')

In [ ]:

# Define the input structure IDs for the textarea
input_test_31_af_ids = 'A0A068N621, A0A0F6AZI6, A0A292DHH8, A0A2U3D0N8, A0A3F2YM30, A0A5H1ZR49, G8ZFK7, O60232, P0A6G5, P0DUH5, P37659, P38164, Q03760, Q08281, Q2K0Z2, Q2UFA9, Q5W0Q7, Q66K64, Q68EN5,Q6CXX6, Q7MVV4, Q86T03, Q8N8R7, Q8NBJ9, Q96JC1, Q9BWG6,Q9D1N4, Q9KP27, Q9M1V3, Q9NUN7, Q9NXF7'

# Textarea for inputting structure IDs
structure_ids_input = widgets.Textarea(
    value=f"{input_test_31_af_ids}",
    description='',
    layout=widgets.Layout(width='100%', height='100px')
)

# Function to save uploaded files to Google Drive
def save_uploaded_files_to_drive(change):
    target_dir = '/content/drive/MyDrive/loaded_strucs'
    os.makedirs(target_dir, exist_ok=True)

    # Save each uploaded file to the target directory
    for file_name, file_info in change['new'].items():
        file_path = os.path.join(target_dir, file_name)
        with open(file_path, 'wb') as f:
            f.write(file_info['content'])
    print(f"Files successfully uploaded to {target_dir}")

# Create a FileUpload widget for file uploads
file_upload = widgets.FileUpload(
    accept='.mmcif,.pdb,.tar,.tar.gz,.txt',  # Restrict accepted file types
    multiple=True,  # Allow multiple file uploads
    description='Upload files'
)

# Automatically save files to Google Drive when uploaded
file_upload.observe(save_uploaded_files_to_drive, names='value')

# Create an Accordion to organize the widgets
accordion = widgets.Accordion(
    children=[structure_ids_input, file_upload],
    selected_index=None  # Keep all sections collapsed by default
)

# Set titles for the accordion sections
accordion.set_title(0, 'Write structure IDs (separated by commas)')
accordion.set_title(1, 'Upload structure files (.mmcif, .pdb, .tar, .tar.gz, .txt)')

# Display the accordion
display(accordion)


Accordion(children=(Textarea(value='A0A068N621, A0A0F6AZI6, A0A292DHH8, A0A2U3D0N8, A0A3F2YM30, A0A5H1ZR49, G8…